In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [ ]:
import shutil

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/output6.zip'

# Specify the destination folder for the extracted files
extracted_folder_path = '/content/output'

# Unzip the file
shutil.unpack_archive(zip_file_path, extracted_folder_path, format='zip')



In [ ]:
shutil.unpack_archive('/content/mlruns.zip', '/content/mlruns', format='zip')

In [ ]:
'''import shutil

# Replace 'path/to/your/folder' with the path to the folder you want to zip
folder_path = '/content/mlruns'

# Specify the name for the zip file (including the full path)
zip_file_name = '/content/mlruns'

# Create a zip file
shutil.make_archive(zip_file_name, 'zip', folder_path)'''

"import shutil\n\n# Replace 'path/to/your/folder' with the path to the folder you want to zip\nfolder_path = '/content/mlruns'\n\n# Specify the name for the zip file (including the full path)\nzip_file_name = '/content/mlruns'\n\n# Create a zip file\nshutil.make_archive(zip_file_name, 'zip', folder_path)"

In [ ]:
!pip install pyngrok

In [ ]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [ ]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist
mlflow.set_experiment("duration-prediction-experiment2")

2024/02/10 08:33:36 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/02/10 08:33:36 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/content/mlruns/1', creation_time=1707554018119, experiment_id='1', last_update_time=1707554018119, lifecycle_stage='active', name='duration-prediction-experiment2', tags={}>

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow

# Initialize Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Start MLflow run
with mlflow.start_run(run_name="LinearRegression_PCA"):

    # Read the Parquet file into a DataFrame
    df_sample = spark.read.parquet("/content/output")
    df_read_parquet = df_sample.limit(1000)

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

    # Create separate columns for each dimension of the embeddings
    for i in range(embedding_length):
        col_name = f"embedding_{i + 1}"
        df_sample = df_sample.withColumn(col_name, df_sample["embeddings"][i])

    # Select embedding variables and 'Impact' for modeling
    feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
    df_model = df_sample.select(feature_columns + ['Impact'])

    # Use PCA for dimensionality reduction to 50 components
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    pca = PCA(k=50, inputCol="features", outputCol="pca_features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

    # Create a Linear Regression model
    lr = LinearRegression(labelCol="Impact", featuresCol="pca_features")

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, pca, lr])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "regParam": cv_model.bestModel.stages[2].getRegParam(),
        "elasticNetParam": cv_model.bestModel.stages[2].getElasticNetParam()
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on training set
    print(f"MAPE on training set: {training_mape:.2f}%")

    # Show MAPE on test set
    print(f"MAPE on test set: {test_mape:.2f}%")

MAPE on training set: 5.32%
MAPE on test set: 5.77%


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from pyngrok import ngrok, conf
import getpass
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


WARNI [pyngrok.process.ngrok] t=2024-02-10T08:37:50+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


 * ngrok tunnel "https://b2d8-35-196-132-5.ngrok-free.app" -> "http://127.0.0.1:5000"


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("BookDataPreprocessing").getOrCreate()

# Read data from CSV file
file_path = "/content/books_task.csv"
df_read_csv = spark.read.csv(file_path, header=True, inferSchema=True)

# Display missing value count before operations
print("Missing Value Count Before Operations:")
df_read_csv.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_read_csv.columns]).show()

# Step 1: Drop rows where 'Title' is empty
df_read_csv = df_read_csv.na.drop(subset=['Title'])

# Step 2: Replace empty 'Authors' with an empty string
df_read_csv = df_read_csv.withColumn('Authors', F.when(df_read_csv['Authors'].isNull(), '').otherwise(df_read_csv['Authors']))

# Step 3: Replace empty 'Publisher' with 'missing'
df_read_csv = df_read_csv.withColumn('Publisher', F.when(df_read_csv['Publisher'].isNull(), 'missing').otherwise(df_read_csv['Publisher']))

# Step 4: Replace empty 'Categories' with an empty string
df_read_csv = df_read_csv.withColumn('Categories', F.when(df_read_csv['Categories'].isNull(), '').otherwise(df_read_csv['Categories']))

# Step 5: Drop rows where 'Impact' is empty
df_read_csv = df_read_csv.na.drop(subset=['Impact'])

# Display missing value count after operations
print("Missing Value Count After Operations:")
df_read_csv.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_read_csv.columns]).show()

# Display data types after operations
print("Data Types After Operations:")
df_read_csv.dtypes

# Show the preprocessed data
df_read_csv.show()


Missing Value Count Before Operations:
+---+-----+-----------+-------+---------+-------------+----------+------+
|_c0|Title|description|authors|publisher|publishedDate|categories|Impact|
+---+-----+-----------+-------+---------+-------------+----------+------+
|  0|    0|       3853|    819|       10|           96|         9|     9|
+---+-----+-----------+-------+---------+-------------+----------+------+

Missing Value Count After Operations:
+---+-----+-----------+-------+---------+-------------+----------+------+
|_c0|Title|description|Authors|Publisher|publishedDate|Categories|Impact|
+---+-----+-----------+-------+---------+-------------+----------+------+
|  0|    0|       3853|      0|        0|           94|         0|     0|
+---+-----+-----------+-------+---------+-------------+----------+------+

Data Types After Operations:
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [ ]:
df_sample.show(1)

+------+--------------------+--------------------+-----------------+---------------+-------------+--------------------+-----------------+--------------------+--------------------+
| Index|               Title|         description|          authors|      publisher|publishedDate|          categories|           Impact|                text|          embeddings|
+------+--------------------+--------------------+-----------------+---------------+-------------+--------------------+-----------------+--------------------+--------------------+
|132341|Shanna's Princess...|Preschoolers will...|['Jean Marzollo']|Jump At The Sun|   2003-04-01|['Juvenile Fiction']|799.1626097737924|Shanna's Princess...|[0.19524505734443...|
+------+--------------------+--------------------+-----------------+---------------+-------------+--------------------+-----------------+--------------------+--------------------+
only showing top 1 row



In [ ]:
df_sample.select('authors').show(1)

+-------+
|authors|
+-------+
|  [[]"]|
+-------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import split, expr

df_sample = df_sample.withColumn("authors", expr("transform(split(substring(authors, 3, length(authors)-4), ','), x -> trim(x, ' []\"'))"))


In [ ]:
df_sample.schema['authors'].dataType

ArrayType(StringType,true)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow

# Initialize Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Start MLflow run
with mlflow.start_run():

    # Read the Parquet file into a DataFrame
    df_read_parquet = spark.read.parquet("/content/output")

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))
    df_read_parquet = df_sample#.limit(10000)

    # Create separate columns for each dimension of the embeddings
    for i in range(embedding_length):
        col_name = f"embedding_{i + 1}"
        df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

    # Select embedding variables and 'Impact' for modeling
    feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
    feature_columns += ['Impact']

    df_model = df_read_parquet.select(feature_columns)

    # Use PCA for dimensionality reduction to 50 components
    pca = PCA(k=50, inputCol="features", outputCol="pca_features")
    assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

    # Create a Linear Regression model
    lr = LinearRegression(labelCol="Impact", featuresCol="pca_features")

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, pca, lr])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "regParam": cv_model.bestModel.stages[2].getRegParam(),
        "elasticNetParam": cv_model.bestModel.stages[2].getElasticNetParam()
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on training set
    print(f"MAPE on training set: {training_mape:.2f}%")

    # Show MAPE on test set
    print(f"MAPE on test set: {test_mape:.2f}%")


WARNI [pyngrok.process.ngrok] t=2024-02-10T08:15:41+0000 lvl=warn msg="failed to open private leg" id=91536b214112 privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"
WARNI [pyngrok.process.ngrok] t=2024-02-10T08:15:42+0000 lvl=warn msg="failed to open private leg" id=de411397f625 privaddr=localhost:5000 err="dial tcp 127.0.0.1:5000: connect: connection refused"


MAPE on training set: 5.32%
MAPE on test set: 5.77%


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow
import mlflow.spark

# Initialize Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Read the Parquet file into a DataFrame
df_sample = spark.read.parquet("/content/output")
df_read_parquet = df_sample#.limit(10000)

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Create a Linear Regression model
lr = LinearRegression(labelCol="Impact", featuresCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Define a parameter grid for tuning
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

# Create a cross-validator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="Impact"),
                          numFolds=3)

# Fit the model
cv_model = crossval.fit(train_df)

# Start MLflow run
with mlflow.start_run():

    # Log parameters and model
    mlflow.log_params({"regParam": cv_model.bestModel.stages[1]._java_obj.getRegParam()})

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Predict on the training set
    train_predictions = cv_model.transform(train_df)

    # Predict on the test set
    test_predictions = cv_model.transform(test_df)

    # Function to calculate MAPE
    def mape(actual_col, forecast_col):
        return F.mean(F.abs((actual_col - forecast_col) / actual_col)) * 100

    # Calculate MAPE for training set
    train_mape = train_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
    mlflow.log_metric("MAPE_train", train_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
    mlflow.log_metric("MAPE_test", test_mape)


In [ ]:
# Predict on the training set
train_predictions = cv_model.transform(train_df)

# Predict on the test set
test_predictions = cv_model.transform(test_df)

# Function to calculate MAPE
def mape(actual_col, forecast_col):
    return F.mean(F.abs((actual_col - forecast_col) / actual_col)) * 100

# Calculate MAPE for training set
train_mape = train_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
print(f"MAPE on training set: {train_mape:.2f}%")

# Calculate MAPE for test set
test_mape = test_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
print(f"MAPE on test set: {test_mape:.2f}%")


MAPE on training set: 4.80%
MAPE on test set: 8.56%


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Read the Parquet file into a DataFrame

df_sample = spark.read.parquet("/content/output")
df_read_parquet = df_sample.limit(1000)

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Create a Linear Regression model
lr = LinearRegression(labelCol="Impact", featuresCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Define a parameter grid for tuning
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

# Create a cross-validator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="Impact"),
                          numFolds=3)

# Fit the model
cv_model = crossval.fit(train_df)

# Predict on the training set
train_predictions = cv_model.transform(train_df)

# Predict on the test set
test_predictions = cv_model.transform(test_df)

# Function to calculate MAPE
def mape(actual_col, forecast_col):
    return F.mean(F.abs((actual_col - forecast_col) / actual_col)) * 100

# Calculate MAPE for training set
train_mape = train_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
print(f"MAPE on training set: {train_mape:.2f}%")

# Calculate MAPE for test set
test_mape = test_predictions.select(mape(F.col('Impact'), F.col('prediction'))).collect()[0][0]
print(f"MAPE on test set: {test_mape:.2f}%")


MAPE on training set: 4.80%
MAPE on test set: 8.56%


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Read the Parquet file into a DataFrame
df_read_parquet = spark.read.parquet("/content/output")

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Show the DataFrame
df_read_parquet.show(truncate=False)

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Check the schema after conversion
# df_read_parquet.printSchema()

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Check the updated schema
df_read_parquet.printSchema()

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Check the statistics of the 'Impact' column
df_model.describe('Impact').show()

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Create a Linear Regression model
lr = LinearRegression(labelCol="Impact", featuresCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Define a parameter grid for tuning
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

# Create a cross-validator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="Impact"),
                          numFolds=3)

# Fit the model
cv_model = crossval.fit(train_df)

# Predict on the training set
train_predictions = cv_model.transform(train_df)

# Predict on the test set
test_predictions = cv_model.transform(test_df)

def smape(actual_col, forecast_col):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE).
    :param actual_col: Column containing actual values
    :param forecast_col: Column containing forecasted values
    :return: Column containing SMAPE
    """
    diff = F.abs(actual_col - forecast_col)
    denominator = (F.abs(actual_col) + F.abs(forecast_col)) / 2.0
    return (diff / denominator) * 100

# Add the SMAPE calculation to your DataFrame
train_predictions = train_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))
test_predictions = test_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))

# Show SMAPE on training set
training_smape = train_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on training set: {training_smape:.2f}%")

# Show SMAPE on test set
test_smape = test_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on test set: {test_smape:.2f}%")

+-----+----------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
'''from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Initialize Spark session with increased memory settings
spark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()

# Read the Parquet file into a DataFrame
df_read_parquet = spark.read.parquet("/content/output")

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Cache the DataFrames for reuse
train_df.cache()
test_df.cache()

# Create a DecisionTreeRegressor model
dt = DecisionTreeRegressor(labelCol="Impact", featuresCol="features")

# Create a pipeline
dt_pipeline = Pipeline(stages=[assembler, dt])

# Define parameter grid for tuning
dt_paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10, 15]).build()

# Create a cross-validator
dt_crossval = CrossValidator(estimator=dt_pipeline,
                             estimatorParamMaps=dt_paramGrid,
                             evaluator=RegressionEvaluator(labelCol="Impact"),
                             numFolds=3)

# Fit the model
dt_model = dt_crossval.fit(train_df)

# Predict on the training set
dt_train_predictions = dt_model.transform(train_df)

# Predict on the test set
dt_test_predictions = dt_model.transform(test_df)

# Function to calculate SMAPE
def smape(actual_col, forecast_col):
    diff = F.abs(actual_col - forecast_col)
    denominator = (F.abs(actual_col) + F.abs(forecast_col)) / 2.0
    return (diff / denominator) * 100

# Add SMAPE calculation to DataFrames
dt_train_predictions = dt_train_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))
dt_test_predictions = dt_test_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))

# Show SMAPE on training set
dt_training_smape = dt_train_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on training set (DecisionTree): {dt_training_smape:.2f}%")

# Show SMAPE on test set
dt_test_smape = dt_test_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on test set (DecisionTree): {dt_test_smape:.2f}%")

# Unpersist the DataFrames to release cache
train_df.unpersist()
test_df.unpersist()

'''

'from pyspark.ml.feature import VectorAssembler\nfrom pyspark.ml.regression import DecisionTreeRegressor\nfrom pyspark.ml import Pipeline\nfrom pyspark.ml.evaluation import RegressionEvaluator\nfrom pyspark.ml.tuning import ParamGridBuilder, CrossValidator\nfrom pyspark.sql import SparkSession\nfrom pyspark.sql.functions import col\nimport pyspark.sql.functions as F\n\n# Initialize Spark session with increased memory settings\nspark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()\n\n# Read the Parquet file into a DataFrame\ndf_read_parquet = spark.read.parquet("/content/output")\n\n# Assuming you have a variable named embedding_length\nembedding_length = 384  # Replace with your actual embedding length\n\n# Check if \'Impact\' is float and convert it to double\ndf_read_parquet = df_read_parquet.withColumn(\'Impact\', col(\'Impact\').cast(\'double\'))\n\n# Create separate columns for each dimension of 

In [ ]:
'''from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Initialize Spark session with increased memory settings
spark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()

# Read the Parquet file into a DataFrame
df_read_parquet = spark.read.parquet("/content/output")

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Cache the DataFrames for reuse
train_df.cache()
test_df.cache()

# Create a RandomForestRegressor model
rf = RandomForestRegressor(labelCol="Impact", featuresCol="features", numTrees=100)

# Create a pipeline with the RandomForestRegressor
rf_pipeline = Pipeline(stages=[assembler, rf])

# Define parameter grid for tuning
rf_paramGrid = ParamsGridBuilder().addGrid(rf.maxDepth, [5, 10, 15]).build()

# Create a cross-validator for Random Forest
rf_crossval = CrossValidator(estimator=rf_pipeline,
                              estimatorParamMaps=rf_paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

# Fit the Random Forest model
rf_model = rf_crossval.fit(train_df)

# Predict on the training set with Random Forest
rf_train_predictions = rf_model.transform(train_df)

# Predict on the test set with Random Forest
rf_test_predictions = rf_model.transform(test_df)

# Function to calculate SMAPE
def smape(actual_col, forecast_col):
    diff = F.abs(actual_col - forecast_col)
    denominator = (F.abs(actual_col) + F.abs(forecast_col)) / 2.0
    return (diff / denominator) * 100

# Add SMAPE calculation to Random Forest predictions
rf_train_predictions = rf_train_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))
rf_test_predictions = rf_test_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))

# Show SMAPE on training set for Random Forest
rf_training_smape = rf_train_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on training set (RandomForest): {rf_training_smape:.2f}%")

# Show SMAPE on test set for Random Forest
rf_test_smape = rf_test_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on test set (RandomForest): {rf_test_smape:.2f}%")

# Unpersist the DataFrames to release cache
train_df.unpersist()
test_df.unpersist()
'''

'from pyspark.ml.feature import VectorAssembler\nfrom pyspark.ml.regression import RandomForestRegressor\nfrom pyspark.ml import Pipeline\nfrom pyspark.ml.evaluation import RegressionEvaluator\nfrom pyspark.ml.tuning import ParamGridBuilder, CrossValidator\nfrom pyspark.sql import SparkSession\nfrom pyspark.sql.functions import col\nimport pyspark.sql.functions as F\n\n# Initialize Spark session with increased memory settings\nspark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").getOrCreate()\n\n# Read the Parquet file into a DataFrame\ndf_read_parquet = spark.read.parquet("/content/output")\n\n# Assuming you have a variable named embedding_length\nembedding_length = 384  # Replace with your actual embedding length\n\n# Check if \'Impact\' is float and convert it to double\ndf_read_parquet = df_read_parquet.withColumn(\'Impact\', col(\'Impact\').cast(\'double\'))\n\n# Create separate columns for each dimension of 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
import pyspark.sql.functions as F

spark = SparkSession.builder.appName("YourAppName").config("spark.executor.memory", "8g").config("spark.driver.memory", "4g").config("spark.executor.cores", "4").config("spark.driver.cores", "2").getOrCreate()

# Read the Parquet file into a DataFrame
df_sample= spark.read.parquet("/content/output")
df_read_parquet = df_sample.limit(1000)

# Assuming you have a variable named embedding_length
embedding_length = 384  # Replace with your actual embedding length

# Check if 'Impact' is float and convert it to double
df_read_parquet = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

# Create separate columns for each dimension of the embeddings
for i in range(embedding_length):
    col_name = f"embedding_{i + 1}"
    df_read_parquet = df_read_parquet.withColumn(col_name, df_read_parquet["embeddings"][i])

# Select embedding variables and 'Impact' for modeling
feature_columns = [f"embedding_{i + 1}" for i in range(embedding_length)]
feature_columns += ['Impact']

df_model = df_read_parquet.select(feature_columns)

# Train-test split
train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

# Prepare feature vector
assembler = VectorAssembler(inputCols=feature_columns[:-1], outputCol="features")

# Cache the DataFrames for reuse
train_df.cache()
test_df.cache()

# Create a RandomForestRegressor model
rf = RandomForestRegressor(labelCol="Impact", featuresCol="features", numTrees=100)

# Create a pipeline with the RandomForestRegressor
rf_pipeline = Pipeline(stages=[assembler, rf])

# Define parameter grid for tuning
rf_paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [5, 10, 15]).build()

# Create a cross-validator for Random Forest
rf_crossval = CrossValidator(estimator=rf_pipeline,
                              estimatorParamMaps=rf_paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

# Fit the Random Forest model
rf_model = rf_crossval.fit(train_df)

# Predict on the training set with Random Forest
rf_train_predictions = rf_model.transform(train_df)

# Predict on the test set with Random Forest
rf_test_predictions = rf_model.transform(test_df)

# Function to calculate SMAPE
def smape(actual_col, forecast_col):
    diff = F.abs(actual_col - forecast_col)
    denominator = (F.abs(actual_col) + F.abs(forecast_col)) / 2.0
    return (diff / denominator) * 100

# Add SMAPE calculation to Random Forest predictions
rf_train_predictions = rf_train_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))
rf_test_predictions = rf_test_predictions.withColumn('smape', smape(F.col('Impact'), F.col('prediction')))

# Show SMAPE on training set for Random Forest
rf_training_smape = rf_train_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on training set (RandomForest): {rf_training_smape:.2f}%")

# Show SMAPE on test set for Random Forest
rf_test_smape = rf_test_predictions.select(F.mean('smape')).collect()[0][0]
print(f"SMAPE on test set (RandomForest): {rf_test_smape:.2f}%")

# Unpersist the DataFrames to release cache
train_df.unpersist()
test_df.unpersist()

KeyboardInterrupt: 

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, expr, coalesce, current_date, year, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from sentence_transformers import SentenceTransformer
from pyspark.sql.types import ArrayType, StringType
from textblob import TextBlob
from pyspark.sql.functions import size, split, length, array_contains



In [ ]:
csv_path = "books_task.csv"
spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4").getOrCreate()

df = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv(csv_path)


In [ ]:
df.count()

138724

In [ ]:
'''# Create a Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").getOrCreate()

# Read data directly into DataFrame with appropriate CSV options
csv_path = "books_task.csv"
df = spark.read.option("header", "true").option("quote", "\"").option("escape", "\"").csv(csv_path)
'''
df.printSchema()

# Rename the '_c0' column to 'Index'
df = df.withColumnRenamed("_c0", "Index")

# Convert 'publishedDate' to date format
df = df.withColumn("publishedDate", col("publishedDate").cast("date"))

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")

# Replace missing values with a default date
default_date = "1900-01-01"
df = df.withColumn("publishedDate", when(col("publishedDate").isNull(), default_date).otherwise(col("publishedDate")))

df.printSchema()

# Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")


# Define a UDF to convert string representation of list to actual list
@udf(ArrayType(StringType()))
def convert_to_list(authors_str):
    return (authors_str[1:-1].replace("'", "").split(',')) if authors_str is not None else []

# Apply the UDF to the 'authors' column
df = df.withColumn("authors_list", convert_to_list(col("authors")))

# Apply the UDF to the 'categories' column
df = df.withColumn("categories_list", convert_to_list(col("categories")))


#Count missing values in each column
missing_counts = [df.where(col(c).isNull()).count() for c in df.columns]

# Create a dictionary to map column names to missing value counts
missing_dict = dict(zip(df.columns, missing_counts))

# Display the missing value counts for each column
for column, missing_count in missing_dict.items():
    print(f"Column '{column}': {missing_count} missing values")

df.printSchema()

# Extract all unique authors
all_authors = df.select("authors_list").rdd.flatMap(lambda x: x[0]).distinct().collect()

# Apply the MultiHot Encoding UDF
print(len(all_authors))


# Print unique publishers
unique_publishers = df.select("publisher").distinct().rdd.flatMap(lambda x: x).collect()
print(len(unique_publishers))


df = df.withColumn("text", coalesce(col("Title"), col("description")))


# Assuming df is your DataFrame
df = df.withColumn("publishedYear", year("publishedDate"))

# calculate the age
df = df.withColumn("age", year(current_date()) - year("publishedDate"))


df = df.withColumn("Impact", col("Impact").cast("double"))




embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def embed_sentences(text):
    embeddings = embeddings_model.encode([text])
    return embeddings.tolist()[0]

# Register the UDF
spark.udf.register("embed_sentences", embed_sentences, "array<double>")

# Apply the UDF to create a new column with embeddings
df = df.withColumn("text_embeddings", expr("embed_sentences(text)"))


embedding_length= 384

# Create separate columns for each dimension of the text_embeddings
for i in range(embedding_length):
        col_name = f"text_embeddings{i + 1}"
        df = df.withColumn(col_name, df["text_embeddings"][i])

all_categories = df.select("categories_list").rdd.flatMap(lambda x: x[0]).distinct().collect()



for i in range(len(all_categories)):
    # Use the 'when' function to create a binary column
    df = df.withColumn(f"category_{i}", when(array_contains(col("categories_list"), all_categories[i]), 1).otherwise(0))

df.show(2)


df = df.withColumn("title_word_count", size(split(col("Title"), " ")))
df = df.withColumn("title_avg_word_length", length(col("Title")) / col("title_word_count"))
df = df.withColumn("char_count", length(col("Title")))
df = df.withColumn("contains_digits", when(col("Title").rlike("\\d"), 1).otherwise(0))
df = df.withColumn("contains_uppercase", when(col("Title").rlike("[A-Z]"), 1).otherwise(0))
df = df.withColumn("contains_punctuation", when(col("Title").rlike("[^\w\s]"), 1).otherwise(0))


# Define a UDF for sentiment analysis using TextBlob
def analyze_sentiment(title):
    blob = TextBlob(title)
    return blob.sentiment.polarity

sentiment_udf = udf(analyze_sentiment, StringType())

# Apply the UDF to the 'Title' column
df = df.withColumn("sentiment_score", sentiment_udf(col("Title")))

df = df.withColumn("sentiment_score", col("sentiment_score").cast("double"))



root
 |-- _c0: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title': 0 missing values
Column 'description': 12749 missing values
Column 'authors': 2723 missing values
Column 'publisher': 0 missing values
Column 'publishedDate': 393 missing values
Column 'categories': 0 missing values
Column 'Impact': 0 missing values
root
 |-- Index: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- Impact: string (nullable = true)

Column 'Index': 0 missing values
Column 'Title'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

+-----+--------------------+--------------------+----------------+--------------------+-------------+--------------------+-----------------+--------------+--------------------+--------------------+-------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------

In [ ]:
columns_to_drop = ['description','categories_list', 'categories','publishedDate', 'publishedYear','authors_list', 'authors','Title','text','text_embeddings']
df = df.drop(*columns_to_drop)

df.write.parquet("data4")

In [ ]:
import shutil

# Replace 'path/to/your/folder' with the path to the folder you want to zip
folder_path = 'data4'

# Specify the name for the zip file (including the full path)
zip_file_name = 'data4'

# Create a zip file
shutil.make_archive(zip_file_name, 'zip', folder_path)

'/content/data4.zip'

In [ ]:
from google.colab import files
files.download('data4.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_sample = spark.read.parquet("data4")


In [ ]:
df_sample.count()

In [ ]:
df_sample.show()

+-----+--------------------+-----------------+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----